In [8]:
import pandas as pd

In [9]:
def read_jsonl(filename):
    df = pd.read_json(filename,lines=True)
    df = df.reset_index(drop=True)
    return df


In [10]:
import os

jsonl_directory = "/media/monlamai/SSD/staging/"

files = os.listdir(jsonl_directory)

pattern = "jsonl"
files_matching_pattern = [file for file in files if file.endswith(pattern)]

In [11]:
#all reviewed files 
reviewed = []
for file in files_matching_pattern:
    if 'review' in file and not 'stt_second_review' in file:
        reviewed.append(file)

In [12]:
#all unreviewed files
transcribed = []
for file in files_matching_pattern:
    if 'review' not in file and not 'stt_second_review' in file:
        transcribed.append(file)

In [13]:
#concatenate all the dataframes
temp= pd.DataFrame([])
for file in transcribed :
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    trabscribed_df = temp.reset_index(drop=True)

In [14]:
#concatenate all the dataframes
temp= pd.DataFrame([])
for file in reviewed:
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    reviewed_df = temp.reset_index(drop=True)

In [15]:
reviewed = temp

In [16]:
finale_df = read_jsonl(f"{jsonl_directory}/stt_second_review.jsonl")

In [17]:
trabscribed_df['id'] = trabscribed_df['id'].str.replace('.mp3','')
trabscribed_df['id'] = trabscribed_df['id'].str.replace('.wav','')
trabscribed_df['id'] = trabscribed_df['id'].str.replace('.MP3','')
trabscribed_df['id'] = trabscribed_df['id'].str.replace('.WAV','')

reviewed_df['id'] = reviewed_df['id'].str.replace('.mp3','')
reviewed_df['id'] = reviewed_df['id'].str.replace('.wav','')
reviewed_df['id'] = reviewed_df['id'].str.replace('.MP3','')
reviewed_df['id'] = reviewed_df['id'].str.replace('.WAV','')

finale_df['id'] = finale_df['id'].str.replace('.mp3','')
finale_df['id'] = finale_df['id'].str.replace('.wav','')
finale_df['id'] = finale_df['id'].str.replace('.MP3','')
finale_df['id'] = finale_df['id'].str.replace('.WAV','')

In [18]:
trabscribed_df['id'].fillna(trabscribed_df['text'], inplace=True)
reviewed_df['id'].fillna(reviewed_df['text'], inplace=True)
finale_df['id'].fillna(finale_df['text'], inplace=True)

In [19]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

(515579, 468714, 150222)

In [20]:
len(set(trabscribed_df['id'])), len(set(reviewed_df['id'])), len(set(finale_df['id']))

(446421, 425269, 146757)

In [21]:
100 - len(set(trabscribed_df['id'])) / len(trabscribed_df) * 100

13.413657266878602

In [22]:
trabscribed_df.drop_duplicates(subset='id', keep="first", inplace=True)
reviewed_df.drop_duplicates(subset='id', keep="first", inplace=True)
finale_df.drop_duplicates(subset='id', keep="first", inplace=True)

In [23]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

(446421, 425269, 146757)

In [24]:
#left side intersection 
intersection = trabscribed_df.merge(reviewed_df, how='left', on='id')

In [25]:
intersection['transcript_y'].fillna(intersection['transcript_x'], inplace=True)


In [26]:
intersection = intersection[~intersection['transcript_y'].isna()]

In [27]:
len(intersection['id']), len(trabscribed_df['id'])

(437285, 446421)

In [28]:
last_intersection = intersection.merge(finale_df, how='left', on='id')

In [29]:
last_intersection.count()

id                   437285
audio_x              437285
url_x                 56271
transcript_x         432813
_input_hash_x        437285
_task_hash_x         437285
_view_id_x           437285
audio_spans_x        435654
answer_x             437285
_timestamp_x         431350
_annotator_id_x      437278
_session_id_x        437278
text_x               381014
meta_x               381014
path_x               381021
_is_binary_x         381021
field_rows_x             34
field_label_x            34
field_id_x               34
field_autofocus_x        34
audio_y              415501
text_y               378222
meta_y               376687
path_y               415480
_input_hash_y        415501
_task_hash_y         415501
_is_binary_y         415480
_view_id_y           415501
audio_spans_y        415500
transcript_y         437285
answer_y             415501
_timestamp_y         415484
_annotator_id_y      415501
_session_id_y        415501
field_rows_y         378271
field_label_y       

In [30]:
last_intersection['transcript'].fillna(last_intersection['transcript_y'], inplace=True)

In [31]:
last_intersection.shape

(437285, 59)

In [32]:
last_intersection = last_intersection[~last_intersection['transcript'].isna()]

In [33]:
last_intersection.shape

(437285, 59)

In [34]:
last_intersection = last_intersection[['transcript', 'id']]

In [35]:
last_intersection['url'] = 'https://d38pmlk0v88drf.cloudfront.net/wav/' + last_intersection['id'] + '.wav'

In [36]:
i = 110001
last_intersection.loc[i]['transcript'], last_intersection.loc[i,'url']

('མ་བརྟག་མ་དཔྱད་པའི་ངོ་ནས། ད་ཐ་སྙད་དུ་གྲགས་པ། གྲགས་ཙམ་གྱི་སྒོ་ནས་བཞག་ཡ་ཞིག་མ་གཏོགས།',
 'https://d38pmlk0v88drf.cloudfront.net/wav/STT_TT00030_02684.600-02689.300.wav')

In [37]:
last_intersection['dept'] = last_intersection['id'].str[:6]

In [38]:
last_intersection.head()

,transcript,id,url,dept
0,དུང་དཀར་རིན་པོ་ཆེས་བླ་མ་དེར་ཕྱག་དཔེའི་ཆ་རྐྱེན་...,STT_AB00001_0041_209714_to_221712,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
1,དུང་དཀར་རིན་པོ་ཆེ,STT_AB00001_0042_222505_to_223855,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
2,རིན་པོ་ཆེ། ཁྱེད་ལ་སློབ་སྦྱོང་གི་ཆ་རྐྱེན་ཤིན་ཏ...,STT_AB00001_0043_224564_to_229795,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
3,ཅེས་གསུངས་པ་ན,STT_AB00001_0044_230453_to_231179,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
4,༄༅།། དུང་དཀར་རིན་པོ་ཆེ་ཆུང་དུས་ནས་བླ་མའི་ཡང་སྤ...,STT_AB00001_0031_154701_to_159021,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB


In [39]:
last_intersection.groupby('dept').count()

,transcript,id,url
dept,,,
STT_AB,56271,56271,56271
STT_CS,36082,36082,36082
STT_NS,45334,45334,45334
STT_TT,299598,299598,299598


In [40]:
import pyewts

converter = pyewts.pyewts()

last_intersection['wylie'] = last_intersection['transcript'].apply(converter.toWylie)

# བ་བྦ་a་བ་

In [41]:
last_intersection

,transcript,id,url,dept,wylie
0,དུང་དཀར་རིན་པོ་ཆེས་བླ་མ་དེར་ཕྱག་དཔེའི་ཆ་རྐྱེན་...,STT_AB00001_0041_209714_to_221712,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,dung dkar rin po ches bla ma der phyag dpe'i c...
1,དུང་དཀར་རིན་པོ་ཆེ,STT_AB00001_0042_222505_to_223855,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,dung dkar rin po che
2,རིན་པོ་ཆེ། ཁྱེད་ལ་སློབ་སྦྱོང་གི་ཆ་རྐྱེན་ཤིན་ཏ...,STT_AB00001_0043_224564_to_229795,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,_rin po che/_khyed la slob sbyong gi cha rkyen...
3,ཅེས་གསུངས་པ་ན,STT_AB00001_0044_230453_to_231179,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,ces gsungs pa na
4,༄༅།། དུང་དཀར་རིན་པོ་ཆེ་ཆུང་དུས་ནས་བླ་མའི་ཡང་སྤ...,STT_AB00001_0031_154701_to_159021,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,@#//_dung dkar rin po che chung dus nas bla ma...
...,...,...,...,...,...
437280,དེ་དག་གི་དཀའ་ཇི་ལྟར་སྐྱེ་བ་,STT_TT00389_240.200-241.250,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,de dag gi dka' ji ltar skye ba
437281,མི་གཤིས་ཀ་དེ་འདྲའི་ཅན་གྱི་མི་ཅིག་ལ་རྟེན་ཅེ་,STT_TT00389_241.800-244.200,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,mi gshis ka de 'dra'i can gyi mi cig la rten ce
437282,ད་ངས་དེ་རིང་འདི་གྲོགས་པོ་བསྒྲིགས་བྱས་ང་ལ་སེམས་...,STT_TT00389_244.600-247.450,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,da ngas de ring 'di grogs po bsgrigs byas nga ...
437283,ང་ལ་ཁེ་བཟང་རག་ཡ་ཅིག་རེ་བ་བརྒྱབ་ཀི་,STT_TT00389_248.100-249.350,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,nga la khe bzang rag ya cig re ba brgyab ki


In [42]:
last_intersection.rename(columns={"id": "file_name", "transcript": "uni", "wylie": "wylie", "url": "url", "dept": "dept"}, inplace=True)

In [43]:
last_intersection = last_intersection[['file_name', 'uni', 'wylie', 'url', 'dept']]
last_intersection.head()

,file_name,uni,wylie,url,dept
0,STT_AB00001_0041_209714_to_221712,དུང་དཀར་རིན་པོ་ཆེས་བླ་མ་དེར་ཕྱག་དཔེའི་ཆ་རྐྱེན་...,dung dkar rin po ches bla ma der phyag dpe'i c...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
1,STT_AB00001_0042_222505_to_223855,དུང་དཀར་རིན་པོ་ཆེ,dung dkar rin po che,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
2,STT_AB00001_0043_224564_to_229795,རིན་པོ་ཆེ། ཁྱེད་ལ་སློབ་སྦྱོང་གི་ཆ་རྐྱེན་ཤིན་ཏ...,_rin po che/_khyed la slob sbyong gi cha rkyen...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
3,STT_AB00001_0044_230453_to_231179,ཅེས་གསུངས་པ་ན,ces gsungs pa na,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
4,STT_AB00001_0031_154701_to_159021,༄༅།། དུང་དཀར་རིན་པོ་ཆེ་ཆུང་དུས་ནས་བླ་མའི་ཡང་སྤ...,@#//_dung dkar rin po che chung dus nas bla ma...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB


In [44]:
last_intersection.to_csv('prodigy.tsv', sep='\t', index=False)

In [45]:
last_intersection[last_intersection['file_name'] == 'STT_AB00001_0029_150584_to_151748']

,file_name,uni,wylie,url,dept
17,STT_AB00001_0029_150584_to_151748,ལེའུ་དང་པོ།\n,_le'u dang po/\n,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB


In [46]:
test = pd.read_csv('staging.tsv', sep='\t')

In [47]:
import pandas as pd

In [48]:
len(test), len(last_intersection)

(437285, 437285)